<a href="https://colab.research.google.com/github/takadeanur/whisper/blob/main/%E3%80%90%E2%98%85%E2%98%8512fps8x%E5%AE%9F%E8%A1%8C%E5%8F%AF%E8%83%BDAPI%E7%89%88%E3%80%91%E9%80%9A%E5%B8%B8%E3%81%AE%E3%82%A2%E3%83%83%E3%83%97%E3%82%B9%E3%82%B1%E3%83%BC%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================
# 0. 初期準備と依存関係のインポート
# ... (変更なし)
# ===============================
import json
import requests
import time
import subprocess
import threading
import os
import shutil
import re
import datetime
import random
from ftplib import FTP
from google.colab import userdata
from typing import List, Tuple

# Colabランタイム切断用モジュールをインポート（最後で利用）
try:
    from google.colab import runtime
except ImportError:
    # Colab外で実行された場合のフォールバック
    class MockRuntime:
        def unassign(self):
            print("🚨 警告: Colab環境外です。runtime.unassign() は実行されません。")
    runtime = MockRuntime()

# ===============================
# 1. 環境準備
# ... (変更なし)
# ===============================
print("--- 1. 環境準備 ---")
# aria2のインストール
!apt -y install -qq aria2

# ===============================
# 2. ComfyUI 本体の取得
# ... (変更なし)
# ===============================
print("\n--- 2. ComfyUI 本体の取得 ---")
%cd /content
!git clone https://github.com/comfyanonymous/ComfyUI
%cd /content/ComfyUI
# 特定のコミットにバージョンを固定
!git reset --hard c60dc4177c16d50da025bda4ac7fd513bb86e699
!pip install -r requirements.txt
!pip install sageattention==1.0.6

# ===============================
# 3. カスタムノードの追加
# ... (変更なし)
# ===============================
print("\n--- 3. カスタムノードの追加 ---")
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
!git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git
!git clone https://github.com/M1kep/ComfyLiterals
!git clone https://github.com/pythongosssss/ComfyUI-Custom-Scripts.git

# ---------------- ComfyUI-Frame-Interpolationの設定 ---------------------
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/Fannovel16/ComfyUI-Frame-Interpolation.git
%cd ComfyUI-Frame-Interpolation
!python install.py # 必要なライブラリやモデルをダウンロード
%cd /content/ComfyUI/custom_nodes
# ---------------- ComfyUI-Frame-Interpolationの設定完了 ---------------------

# ---------------- rgthree-comfyの設定 ---------------------
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/rgthree/rgthree-comfy
%cd rgthree-comfy
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes
# ---------------- rgthree-comfyの設定完了 ---------------------


# ===============================
# 4. モデルのダウンロード
# ... (変更なし)
# ===============================
print("\n--- 4. モデルのダウンロード ---")

# Upscale model (RealESRGAN x4)
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ai-forever/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -d /content/ComfyUI/models/upscale_models -o RealESRGAN_x2.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ai-forever/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -d /content/ComfyUI/models/upscale_models -o RealESRGAN_x4.pth

# =========================================================================
# 5. FTP連携とAPI実行用のPythonコード (ユーザー提供のコードをメインに統合)
# =========================================================================

# --- 5-1. ファイルパスと設定（FTP関連） ---
PORT = 8188
COMFYUI_PORT = PORT
JSON_FILE_NAME = "original_video_workflow.json"
PROMPT_FILE_PATH = f"/content/{JSON_FILE_NAME}"
COMFYUI_SERVER_URL = f"http://127.0.0.1:{COMFYUI_PORT}"

# ★★★ 動画生成の強制再実行フラグ (ここを切り替えます) ★★★
# True: 既存の *_8x.mp4 の有無に関わらず再生成する (再作成モード)
# False: 既存の *_8x.mp4 がない動画のみを生成する (未生成チェックモード)
FORCE_RECREATE = False
# ★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★★
MAX_VIDEOS_TO_PROCESS = 50 # 最大処理件数 (この件数を超えた場合、リストの先頭から処理されます)

# シークレット情報の取得
print("\n--- 5-2. FTP接続情報の取得 ---")
try:
    FTP_HOST = userdata.get('ftp_host')
    FTP_USER = userdata.get('ftp_user')
    FTP_PASS = userdata.get('ftp_pass')
except:
    FTP_HOST = "your_ftp_host"
    FTP_USER = "your_ftp_user"
    FTP_PASS = "your_ftp_pass"


if not (FTP_HOST and FTP_USER and FTP_PASS) or FTP_HOST == "your_ftp_host":
    print("🚨 警告: FTP接続情報が取得できないか、テスト値のままです。FTP機能は動作しません。")
else:
    print("✅ FTP接続情報の取得に成功しました。")

# --- 入力画像のベースディレクトリ設定 ---
FTP_BASE_DIR = "/jitantech.com/public_html/py-project/youtube-news-project"
INPUT_BASE_DIR_NAME = "downloaded_images_by_channel_id_rank"
INPUT_BASE_FTP_FOLDER = f"{FTP_BASE_DIR}/{INPUT_BASE_DIR_NAME}/"

# ワークフローの LoadVideo ノードが参照する固定ファイル名とパス
COMFYUI_INPUT_FILENAME = "video.mp4" # 処理中はこれにリネームして使用
INPUT_IMAGE_LOCAL_PATH = f"/content/ComfyUI/input/{COMFYUI_INPUT_FILENAME}"
LOAD_VIDEO_NODE_ID = "10"
LOAD_VIDEO_INPUT_KEY = "video"


print(f"⚙️ 処理対象の画像ベースフォルダ: {INPUT_BASE_FTP_FOLDER}")
print(f"⚙️ ComfyUI入力ファイル名: {COMFYUI_INPUT_FILENAME}")


# --- 5-3. ワークフローJSONをPythonオブジェクトとして定義 ---
# (ワークフローの内容は変更なし)
workflow = {
    "10": {
        "inputs": {
            "video": COMFYUI_INPUT_FILENAME,
            "force_rate": 0,
            "custom_width": 0,
            "custom_height": 0,
            "frame_load_cap": 0,
            "skip_first_frames": 1,
            "select_every_nth": 1,
            "format": "AnimateDiff"
        },
        "class_type": "VHS_LoadVideo",
        "_meta": {
            "title": "Load Video (Upload) 🎥🅥🅗🅢"
        }
    },
    "13": {
        "inputs": {
            "frame_rate": [
                "34",
                1
            ],
            "loop_count": 0,
            "filename_prefix": "Upscaled",
            "format": "video/h264-mp4",
            "pix_fmt": "yuv420p",
            "crf": 10,
            "save_metadata": True,
            "trim_to_audio": False,
            "pingpong": False,
            "save_output": True,
            "images": [
                "23",
                0
            ],
            "audio": [
                "10",
                2
            ]
        },
        "class_type": "VHS_VideoCombine",
        "_meta": {
            "title": "Video Combine 🎥🅥🅗🅢"
        }
    },
    "23": {
        "inputs": {
            "upscale_model": [
                "24",
                0
            ],
            "image": [
                "27",
                0
            ]
        },
        "class_type": "ImageUpscaleWithModel",
        "_meta": {
            "title": "モデルを使用して画像を拡大"
        }
    },
    "24": {
        "inputs": {
            "model_name": "RealESRGAN_x4.pth"
        },
        "class_type": "UpscaleModelLoader",
        "_meta": {
            "title": "拡大モデルを読み込む"
        }
    },
    "27": {
        "inputs": {
            "ckpt_name": "rife47.pth",
            "clear_cache_after_n_frames": 10,
            "multiplier": [
                "35",
                0
            ],
            "fast_mode": True,
            "ensemble": True,
            "scale_factor": 1,
            "frames": [
                "10",
                0
            ]
        },
        "class_type": "RIFE VFI",
        "_meta": {
            "title": "RIFE VFI (recommend rife47 and rife49)"
        }
    },
    "34": {
        "inputs": {
            "expression": "a * b",
            "a": [
                "40",
                0
            ],
            "b": [
                "35",
                0
            ]
        },
        "class_type": "MathExpression|pysssss",
        "_meta": {
            "title": "Math Expression 🐍"
        }
    },
    "35": {
        "inputs": {
            "Number": "1"
        },
        "class_type": "Int",
        "_meta": {
            "title": "FPS Multipler"
        }
    },
    "40": {
        "inputs": {
            "video_info": [
                "10",
                3
            ]
        },
        "class_type": "VHS_VideoInfo",
        "_meta": {
            "title": "Video Info 🎥🅥🅗🅢"
        }
    }
}

# --- JSON ワークフローを事前に書き出す ---
%cd /content
with open(JSON_FILE_NAME, 'w') as f:
    json.dump(workflow, f, indent=2)
print(f"✅ ワークフローJSONを {PROMPT_FILE_PATH} に書き出しました。")
%cd /content/ComfyUI


# =======================================================
# 6. 動画リスト取得関数 (特定のファイル 'gyang_001_720px.mp4' を全フォルダから取得)
# =======================================================
def get_unprocessed_image_list() -> List[Tuple[str, str]]:
    """
    FTPベースディレクトリ以下の全サブフォルダを検索し、
    各フォルダ内に存在する 'gyang_001_720px.mp4' のうち、
    アップスケール済み（gyang_001_720px_8x.mp4が存在する）でないもの全てをリストアップします。

    戻り値: [(フルパス, ファイル名), ...] のリスト
    """
    # ★★★ ターゲットファイル名を設定 ★★★
    TARGET_VIDEO_FILENAME = "rap_001_720px.mp4"
    # --- 変更点 1: 期待される出力ファイル名を _8x.mp4 に変更 ---
    EXPECTED_OUTPUT_FILENAME = "rap_001_720px_8x.mp4"
    # ★★★★★★★★★★★★★★★★★★★★★★★

    if FORCE_RECREATE:
        print(f"\n⚙️ **強制再生成モード** が有効です。全フォルダの '{TARGET_VIDEO_FILENAME}' を処理対象とします。")
    else:
        print(f"\n⚙️ **未生成チェックモード** が有効です。出力ファイル '{EXPECTED_OUTPUT_FILENAME}' の有無を全フォルダでチェックします。")

    print(f"FTP サーバー ({INPUT_BASE_FTP_FOLDER}) から対象動画を検索します...")
    unprocessed_videos = []
    found_any_target = False

    try:
        with FTP(FTP_HOST) as ftp:
            ftp.login(user=FTP_USER, passwd=FTP_PASS)

            try:
                # すべてのサブフォルダ（チャンネルIDフォルダ）のリストを取得
                sub_dir_full_paths = []
                for item in ftp.nlst(INPUT_BASE_FTP_FOLDER):
                    # 拡張子がないものをフォルダとして扱う
                    if not os.path.splitext(item)[1]:
                         sub_dir_full_paths.append(item)

                processed_sub_dirs = sorted(sub_dir_full_paths)
            except Exception as e:
                raise Exception(f"🚨 FTPベースディレクトリ ({INPUT_BASE_FTP_FOLDER}) へのアクセスに失敗しました: {e}")

            if not processed_sub_dirs:
                print("⚠️ 処理すべきサブフォルダ（チャンネルIDフォルダ）が見つかりませんでした。")
                return []

            print(f"✅ {len(processed_sub_dirs)} 件のサブフォルダを検出しました。")

            for folder_path in processed_sub_dirs:
                if not folder_path.endswith('/'): folder_path += '/'
                log_folder_name = os.path.basename(os.path.normpath(folder_path))

                files_in_source = []
                source_filenames = []
                try:
                    # サブフォルダ内のファイルリストを取得
                    files_in_source = ftp.nlst(folder_path)
                    source_filenames = [os.path.basename(f) for f in files_in_source]
                except:
                    continue # アクセスできないフォルダはスキップ

                # 指定されたターゲットファイルが存在するかチェック
                if TARGET_VIDEO_FILENAME in source_filenames:
                    found_any_target = True
                    full_ftp_path = os.path.join(folder_path, TARGET_VIDEO_FILENAME)

                    if FORCE_RECREATE:
                        # 強制再作成モードの場合、常にリストに追加
                        unprocessed_videos.append((full_ftp_path, TARGET_VIDEO_FILENAME))
                        print(f"➡️ 強制処理対象: フォルダ: {log_folder_name}, ファイル: {TARGET_VIDEO_FILENAME}")
                    elif EXPECTED_OUTPUT_FILENAME not in source_filenames:
                        # 未生成チェックモードの場合、出力ファイルが存在しない場合は処理が必要
                        unprocessed_videos.append((full_ftp_path, TARGET_VIDEO_FILENAME))
                        print(f"➡️ 未生成: フォルダ: {log_folder_name}, ファイル: {TARGET_VIDEO_FILENAME}")
                    else:
                        # 出力ファイルが存在する場合はスキップ
                        print(f"✅ 存在確認OK: フォルダ: {log_folder_name}, ファイル: {EXPECTED_OUTPUT_FILENAME} が存在するためスキップします。")

                # 処理件数に達したら、FTP接続を維持したままループを終了
                if len(unprocessed_videos) >= MAX_VIDEOS_TO_PROCESS:
                    print(f"⚠️ 処理対象の検出が最大処理件数 {MAX_VIDEOS_TO_PROCESS} 件に達しました。")
                    break

            if not found_any_target:
                print(f"🚨 エラー: 全てのフォルダで指定された動画ファイル '{TARGET_VIDEO_FILENAME}' が見つかりませんでした。")
                return []

            if not unprocessed_videos:
                print(f"✅ 全フォルダの '{TARGET_VIDEO_FILENAME}' は既にアップスケール済みのため、処理対象はありません。")
                return []

            mode_text = "強制再生成モード" if FORCE_RECREATE else "未生成チェックモード"
            print(f"✅ {mode_text} で、合計 {len(unprocessed_videos)} 件の動画ファイル '{TARGET_VIDEO_FILENAME}' を処理対象として見つけました。")

            # MAX_VIDEOS_TO_PROCESS でリストをトリミング (ループ内でチェック済みだが念のため)
            if len(unprocessed_videos) > MAX_VIDEOS_TO_PROCESS:
                unprocessed_videos = unprocessed_videos[:MAX_VIDEOS_TO_PROCESS]

            return unprocessed_videos

    except Exception as e:
        raise Exception(f"🚨 FTP (動画リスト取得) 中に致命的なエラーが発生しました: {e}")


# =======================================================
# 7. FTPからの単一動画ダウンロード (変更なし)
# ...
# =======================================================
def download_image_to_comfyui_input(ftp_image_full_path, local_input_path, verbose=True):
    """ 指定されたFTP動画をダウンロードし、ComfyUIの固定入力パスに保存します。"""
    if verbose:
        print(f"⚙️ 入力動画 {ftp_image_full_path} を {local_input_path} にダウンロード中...")
    # ... (処理本体は変更なし) ...
    try:
        with FTP(FTP_HOST) as ftp:
            ftp.login(user=FTP_USER, passwd=FTP_PASS)
            # 古い固定入力ファイルを削除
            if os.path.exists(local_input_path):
                os.remove(local_input_path)
            # ダウンロード
            with open(local_input_path, 'wb') as f:
                ftp.retrbinary(f'RETR {ftp_image_full_path}', f.write)
        if not os.path.exists(local_input_path):
            raise FileNotFoundError(f"FTPダウンロード後、ファイル {local_input_path} が見つかりません。")
        if verbose:
            print(f"✅ ダウンロード完了。ComfyUIの入力ファイル名: {os.path.basename(local_input_path)}")
        return True
    except Exception as e:
        if verbose:
            print(f"🚨 動画ダウンロード中にエラーが発生しました: {e}")
        return False


# =======================================================
# 8. FTPへの動画アップロード (変更なし)
# ...
# =======================================================
def upload_video_to_ftp(local_video_path, ftp_source_image_full_path, generated_video_name):
    """
    生成された動画をFTPサーバー上の、元の動画と同じフォルダにアップロードします。
    """
    if not local_video_path or not os.path.exists(local_video_path):
        print(f"🚨 エラー: 生成動画ファイル {local_video_path} が見つかりません。アップロードをスキップします。")
        return
    # 元動画ファイルのディレクトリパスを取得
    ftp_target_dir = os.path.dirname(ftp_source_image_full_path) + "/"
    remote_path = os.path.join(ftp_target_dir, generated_video_name)
    print(f"\nFTP サーバー ({ftp_target_dir}) へ生成動画 ({generated_video_name}) をアップロードします...")
    # ... (処理本体は変更なし) ...
    try:
        with FTP(FTP_HOST) as ftp:
            ftp.login(user=FTP_USER, passwd=FTP_PASS)
            # アップロード
            with open(local_video_path, 'rb') as file:
                ftp.storbinary(f'STOR {remote_path}', file)
            print(f"✅ 生成動画 {generated_video_name} を元の動画と同じディレクトリにアップロードしました。")
    except Exception as e:
        print(f"🚨 FTP (動画アップロード) 中にエラーが発生しました: {e}")


# =======================================================
# 9. 動画生成 API クライアント関数 (変更なし)
# ...
# =======================================================
def generate_video_with_comfyui(prompt_json_path, server_url, output_video_name):
    """ ローカル ComfyUI API に接続し、動画を生成してローカルファイルパスを返します。"""
    # ... (処理本体は変更なし) ...
    try:
        # 1. ワークフローJSONの読み込み
        with open(prompt_json_path, 'r') as f:
            prompt = json.load(f)
        # 3. ワークフロー実行のリクエスト
        client_id = "colab_client_" + str(os.getpid())
        prompt_url = f"{server_url}/prompt"
        payload = {"prompt": prompt, "client_id": client_id}
        print("\n--- ワークフロー送信 ---")
        max_retries = 3
        response = None
        for attempt in range(max_retries):
            try:
                response = requests.post(prompt_url, json=payload, timeout=5)
                response.raise_for_status()
                break
            except requests.exceptions.RequestException as e:
                if attempt < max_retries - 1:
                    print(f"警告: API接続エラー (試行 {attempt + 1}/{max_retries})。再試行します: {e}")
                    time.sleep(2)
                else:
                    raise e
        # 4. 実行完了のポーリング
        prompt_id = response.json().get('prompt_id')
        print(f"実行がキューに追加されました。Prompt ID: {prompt_id}")
        history_url = f"{server_url}/history/{prompt_id}"
        start_poll_time = time.time()
        max_poll_time = 10800 # 3時間
        while time.time() - start_poll_time < max_poll_time:
            time.sleep(2)
            try:
                history_response = requests.get(history_url)
                history_response.raise_for_status()
                if prompt_id in history_response.json():
                    print("動画生成が完了しました。")
                    break
            except requests.exceptions.RequestException:
                continue
        else:
            print(f"🚨 エラー: 動画生成が {max_poll_time} 秒以内に完了しませんでした。")
            return None
        # 5. 動画ダウンロード (API経由での取得を試みる)
        output_info = history_response.json()[prompt_id]['outputs']
        video_data = output_info.get('13', {}).get('videos', []) or output_info.get('41', {}).get('videos', [])
        if video_data:
            file_info = video_data[0]
            view_url = f"{server_url}/view"
            params = {
                "filename": file_info['filename'],
                "subfolder": file_info['subfolder'],
                "type": file_info['type'],
                "video": "true"
            }
            print(f"APIから動画ファイルをダウンロード中: {file_info['filename']}")
            video_response = requests.get(view_url, params=params)
            video_response.raise_for_status()
            with open(output_video_name, 'wb') as f:
                f.write(video_response.content)
            print(f"✅ 動画をローカルの {output_video_name} に保存しました。(API成功)")
            return output_video_name
        # 6. フォールバック: ローカルの最新ファイルを、output/ と output/video/ から取得する
        comfy_output_dirs = ["/content/ComfyUI/output/", "/content/ComfyUI/output/video/"]
        latest_video_path = None
        print("\n--- ローカルフォールバック処理を開始 (output/video/ 含む) ---")
        for check_dir in comfy_output_dirs:
            if not os.path.exists(check_dir):
                continue
            for file in os.listdir(check_dir):
                if file.endswith(".mp4"):
                    file_path = os.path.join(check_dir, file)
                    if not latest_video_path or os.path.getmtime(file_path) > os.path.getmtime(latest_video_path):
                        latest_video_path = file_path
        if latest_video_path:
            print(f"✅ ローカル出力ディレクトリから動画ファイルを見つけました: {latest_video_path}")
            shutil.copyfile(latest_video_path, output_video_name)
            try:
                os.remove(latest_video_path)
                print(f"🗑️ ローカル出力ファイル {latest_video_path} をクリーンアップしました。")
            except Exception as e:
                print(f"⚠️ クリーンアップに失敗: {e}")
            return output_video_name
        print("🚨 致命的エラー: 動画ファイル情報が見つかりませんでした。動画が生成されていない可能性があります。")
        return None
    except Exception as e:
        print(f"🚨 処理中に予期せぬエラー: {e}")
        return None


# ===============================
# 10. ComfyUIサーバー起動 (変更なし)
# ...
# ===============================
def start_comfyui_server(port):
    print(f"Starting ComfyUI server in API mode on port {port}...")
    # --listenでAPIモードを有効にする
    comfyui_process = subprocess.Popen(
        f"python main.py --listen --port {port}",
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        cwd="/content/ComfyUI"
    )
    return comfyui_process

def wait_for_server(process, port):
    server_url = f"http://127.0.0.1:{port}"
    start_time = time.time()
    max_wait = 60
    print("ComfyUIサーバー起動を待機中...")
    # ... (処理本体は変更なし) ...
    while time.time() - start_time < max_wait:
        try:
            response = requests.get(f"{server_url}/queue", timeout=1)
            if response.status_code == 200:
                print("✅ ComfyUIサーバーの起動を確認しました。")
                return True
        except requests.exceptions.RequestException:
            pass
        # サーバーログをチェック
        line = process.stdout.readline()
        if line:
            print(f"ComfyUI Log: {line.strip()}")
            if "To see the GUI go to:" in line or "Unet loaded" in line or "Running on" in line:
                print("サーバーからのログにより、起動確認をしました。")
                return True
        time.sleep(1)
    print("🚨 エラー: ComfyUIサーバーが60秒以内に起動しませんでした。")
    return False


# ===============================
# 11. 全体実行フロー (全フォルダの指定ファイル処理)
# ===============================
if __name__ == "__main__":

    unprocessed_list = []
    comfyui_process = None

    # 1. FTPから未処理動画のリストを取得 (全フォルダの 'gyang_001_720px.mp4' が対象)
    try:
        # get_unprocessed_image_list内でMAX_VIDEOS_TO_PROCESSの制限が適用されます。
        unprocessed_list = get_unprocessed_image_list()
    except Exception as e:
        print(f"処理を中断します。エラー: {e}")
        exit()

    if not unprocessed_list:
        print("処理対象の動画ファイルが見つからなかったか、既にアップスケール済みです。処理を終了します。")
        exit()

    # 2. ComfyUIサーバーの起動
    print("\n--- ComfyUIサーバー起動 (API専用) ---")
    comfyui_process = start_comfyui_server(COMFYUI_PORT)

    # 3. サーバー起動確認
    if not wait_for_server(comfyui_process, COMFYUI_PORT):
        if comfyui_process:
            comfyui_process.terminate()
        exit()

    # 4. API クライアントを実行して動画を生成 (リストをループ処理)
    processed_count = 0
    total_to_process = len(unprocessed_list)

    print(f"\n--- 合計 {total_to_process} 件の動画に対するループ処理を開始します (最大 {MAX_VIDEOS_TO_PROCESS} 件) ---")

    for i, (input_image_ftp_full_path, input_image_filename) in enumerate(unprocessed_list):

        # 処理件数の上限チェック (リストの取得時に既に制限されているため、主にログ用)
        if processed_count >= MAX_VIDEOS_TO_PROCESS:
            print(f"\n✅ 最大処理件数 {MAX_VIDEOS_TO_PROCESS} 件に達したため、ループを中断します。")
            break # ループを抜ける

        # フォルダ名（フルパス）を取得
        folder_path = os.path.dirname(input_image_ftp_full_path)
        base_filename = os.path.splitext(input_image_filename)[0]
        # output_video_ftp_name は 'gyang_001_720px_8x.mp4' となる
        # --- 変更点 2: 出力ファイル名を _8x.mp4 に変更 ---
        output_video_ftp_name = f"{base_filename}_8x.mp4"
        output_video_name = f"/content/{output_video_ftp_name}" # ローカルの一時保存パス

        print(f"\n\n========================================================")
        print(f"== 処理開始: {i + 1}/{total_to_process} - フォルダ: {os.path.basename(os.path.normpath(folder_path))} ==")
        print(f"➡️ 入力ファイル名: {input_image_filename}")
        print(f"➡️ 出力予定ファイル名: {output_video_ftp_name}")
        print(f"========================================================")


        # 4a. 入力動画をダウンロード (ComfyUIの固定入力パスへ)
        if not download_image_to_comfyui_input(input_image_ftp_full_path, INPUT_IMAGE_LOCAL_PATH, verbose=True):
            continue

        # 4b. API クライアントを実行して動画を生成
        generated_local_path = generate_video_with_comfyui(PROMPT_FILE_PATH, COMFYUI_SERVER_URL, output_video_name)

        # 4c. FTPへ動画をアップロード (元の動画と同じ場所へ、新しいファイル名で)
        if generated_local_path:
            upload_video_to_ftp(generated_local_path, input_image_ftp_full_path, output_video_ftp_name)
            # 処理件数をカウントアップ
            processed_count += 1
            print(f"🔄 現在の処理完了件数: {processed_count}/{total_to_process} (フォルダ: {os.path.basename(os.path.normpath(folder_path))})")

        # 4d. クリーンアップ
        if os.path.exists(INPUT_IMAGE_LOCAL_PATH):
            os.remove(INPUT_IMAGE_LOCAL_PATH)
            # print(f"🗑️ {INPUT_IMAGE_LOCAL_PATH} (ComfyUI入力) を削除しました。")
        if os.path.exists(output_video_name):
            os.remove(output_video_name)
            # print(f"🗑️️ {output_video_name} (ローカル一時ファイル) を削除しました。")


    # 5. 処理終了後、プロセスを終了
    if comfyui_process:
        comfyui_process.terminate()
        print("\nComfyUI プロセスを終了しました。")

    # 6. 最終クリーンアップ
    print("\n--- 最終クリーンアップ ---")
    if os.path.exists(PROMPT_FILE_PATH):
        os.remove(PROMPT_FILE_PATH)
        print(f"🗑️ {PROMPT_FILE_PATH} を削除しました。")


    print("\n--- 全ての処理が完了しました ---")

    # 7. ランタイム自動終了ロジック
    if processed_count >= MAX_VIDEOS_TO_PROCESS:
        print("\n🚨 処理件数上限に達しました。ランタイムを自動的に切断します。")
        # Colabのランタイムを切断（セッションの終了）するコマンド
        runtime.unassign()

--- 1. 環境準備 ---
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 41 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.36.0-1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.36.0-1_amd64.deb ...
Unpacking aria2 (1.36.0-1) ...
Setting up libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Setting up libaria2-0:amd64 (1.36.0-1) ...
Setting up a